# UBC Lost & Found Predictive Analysis

<img src="./img/access.jpg" alt="drawing" style="width:90%;"/>
Access Services. Retrieved from <a href = "https://parking.ubc.ca/">UBC Parking</a> 

## Introduction 
---

Losing personal belongings is a common and often frustrating experience for students and staff at UBC. Items ranging from everyday essentials like wallets and keys to electronics and jewelry frequently end up in the lost and found. While the university maintains a central lost and found system, the lack of systematic analysis in these services can lead to inefficiencies, making it harder to identify trends and optimize recovery processes. By analyzing the UBC lost and found dataset, this project aims to uncover patterns in lost items, predict future trends, and provide actionable insights to enhance the efficiency of UBC's lost and found services.

**Objectives**: 
- **Item Classification Analysis**: Identify the most frequently lost item categories (e.g., electronics, wallets, keys).
- **Temporal Analysis**: Examine seasonal and daily trends in lost item reports.
- **Location Analysis**: Identify the most common campus locations where items are lost and found.
- **Item Description Analysis**: Group similar items based on "Item Description" to identify detailed subcategories of lost items using NLP techniques
- **Time Series Forecasting**: Develop predictive models to forecast the number of items reported to the lost and found using historical data.
- **Recommendations**: Provide actionable recommendations to improve the lost and found system, including targeted awareness campaigns, optimized item storage, and enhanced communication channels.


## Dataset
---

The [UBC Lost and Found database](https://lostandfound.ubc.ca/all-items) is a publicly accessible dataset containing information about items reported lost within the University of British Columbia campus. The dataset spans from May 2024 to the current day, with over 150 entries. Each entry includes details such as the item type, description, date lost, and location. We will be using the dataset retrieved as of January 2025 for the project.

### Loading libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import mglearn
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

### Importing the data

In [2]:
url = "https://lostandfound.ubc.ca/all-items/export.csv"
df = pd.read_csv(url, index_col = "Date")

df.head()

,Ticket Number,Item Type,Status,Item Description,Lost Item Location
Date,,,,,
"January 17, 2025",47182,Jewelry,Lost,Metal Watch,Library
"January 16, 2025",47181,Jewelry,Lost,Two tone Ring,Other
"January 16, 2025",47180,Jewelry,Lost,Plain Ring,Other
"January 16, 2025",47179,Jewelry,Lost,Ring with Stone,Other
"January 16, 2025",47178,Keys,Lost,Single Key with wrist Ring,Other
